Medium Link :- https://keeganfdes03.medium.com/29f040158ef3?source=friends_link&sk=c0367f499bcb04f71db88de26983462b

# Dependencies

In [ ]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torch import nn
import cv2
import pytorch_lightning as pl
import torch
from sklearn.model_selection import train_test_split
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer

In [ ]:
input_df = "../input/images-of-randomly-generated-quadratic-equations/Quadratic Equation Full Details.csv"
df = pd.read_csv(input_df)
df

# Standard Pytorch Dataset

In [ ]:
class Quadratic_Dataset(Dataset):
    def __init__(self,path,targets = None):
        self.path =  path
        self.targets = targets
        self.dir_path = "../input/images-of-randomly-generated-quadratic-equations/Images of Equations/"
    
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, index):
        image_path = self.dir_path + self.path[index] + ".png"
        image = cv2.imread(image_path)
        image = cv2.resize(image,(100,100))
        if self.targets is None:
            return 1.0 - torch.tensor(image).float().reshape(3,100,100)/256
        else:
            return 1.0 - torch.tensor(image).float().reshape(3,100,100)/256 , torch.tensor(self.targets[index]).float().reshape(3)


###  Sanity Check to Make Sure of what you are sending to the model ###

In [ ]:
dataset = Quadratic_Dataset(path = df["id"].values , targets = df[["a","b","c"]].values)
next(iter(DataLoader(dataset , batch_size = 1 , shuffle = True)))

# Preprocessing the Data

In [ ]:
def normalize(df_values,min_,max_):
    df_list = df_values
    return [(x-min_)/(max_ - min_) for x in df_values]

In [ ]:
def denormalize(df_values,min_,max_):
    df_list = df_values
    return [(x*(max_ - min_) + min_) for x in df_values]

In [ ]:
df['a_'] = normalize(df["a"].values , 1 , 20)
df['b_'] = normalize(df["b"].values,-48,96)
df['c_'] = normalize(df["c"].values,-792,600)

In [ ]:
train_df , test_df = train_test_split(df,test_size = 0.2 , random_state= 42)
train_df , val_df = train_test_split(test_df,test_size = 0.33 , random_state= 42)

# Lightning Datamodule

In [ ]:
class Quadratic_Module(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.train_dataset = Quadratic_Dataset(path = train_df["id"].values , targets = train_df[["a_","b_", "c_"]].values)
        self.test_dataset = Quadratic_Dataset(path = test_df["id"].values , targets = test_df[["a_","b_" , "c_"]].values)
        self.val_dataset = Quadratic_Dataset(path = val_df["id"].values , targets = val_df[["a_","b_" , "c_"]].values)
        self.predictions = Quadratic_Dataset(path = test_df["id"].values , targets = None)

    def prepare_data(self) :
        pass
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset , batch_size = 32  , shuffle = True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset , batch_size = 32  , shuffle = False)

    def val_dataloader(self):
        return DataLoader(self.val_dataset , batch_size = 32  , shuffle = False)
    
    def predict_dataloader(self):
        return DataLoader(self.predictions , batch_size = 1 , shuffle  = False)

In [ ]:
neural_network = torchvision.models.resnet50(pretrained = True)
neural_network.fc = nn.Sequential(
    nn.Linear(2048,1024),
    nn.LeakyReLU(0.2),
    nn.Linear(1024,512),
    nn.LeakyReLU(0.2),
    nn.Linear(512,256),
    nn.LeakyReLU(0.2),
    nn.Linear(256,100),
    nn.LeakyReLU(0.2),
    nn.Linear(100,10),
    nn.Linear(10,3),
    
)

### Sanity Check 

In [ ]:
x = torch.ones(1,3,100,100)
neural_network(x)

In [ ]:
class Quadratic_Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.loss_func = nn.MSELoss()
        self.net = neural_network
        
        
    def forward(self,x):
        out = self.net(x)
        return out
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = 1e-3)
        sch = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size  = 10 , gamma = 0.5) #learning raet scheduler
        return {
            "optimizer":optimizer,
            "lr_scheduler" : {
                "scheduler" : sch,
                "monitor" : "train_loss",
                
            }
        }

    def training_step(self,batch,batch_idx):
        x , y_true = batch
        y_pred = self(x)
        loss = self.loss_func(y_true, y_pred)
        self.log("train_loss" , loss)
        return loss
    
    def test_step(self,batch,batch_idx):
        x , y_true = batch
        y_pred = self(x)
        loss = self.loss_func(y_true, y_pred)
        self.log("test_loss" , loss)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x , y_true = batch
        y_pred = self(x)
        loss = self.loss_func(y_true, y_pred)
        self.log("val_loss" , loss)
        return loss



In [ ]:
checkpoint = ModelCheckpoint(monitor= "val_loss" ,mode = "min") #Trainer returns the model with the lowest "val_loss"

In [ ]:
model = Quadratic_Model()
module = Quadratic_Module()
trainer = Trainer(max_epochs = 100,callbacks=[checkpoint] , gpus= 1)

# Training the model

In [ ]:
trainer.fit(model,module)

# Testing the model

In [ ]:
trainer.test()


# Making Predictions

In [ ]:
predictions = trainer.predict() # Makes predictions from the test set
actual_values_a = test_df["a"].values
actual_values_b = test_df["b"].values
actual_values_c = test_df["c"].values

In [ ]:
prediction_a = []
prediction_b = []
prediction_c = []
i = 0
for preds in predictions:  
    preds = [float(x) for x in preds.reshape(3)]
    prediction_a.append(float(preds[0]))
    prediction_b.append(float(preds[1]))
    prediction_c.append(float(preds[2]))
    
prediction_df = pd.DataFrame({
    "predictions_a" : prediction_a,
    "predictions_b" : prediction_b,
    "predictions_c" : prediction_c,
    "actual_values_a" : actual_values_a,
    "actual_values_b" : actual_values_b,
    "actual_values_c" : actual_values_c
})

In [ ]:
prediction_df['predictions_a'] = denormalize(prediction_df["predictions_a"].values , 1 , 20)
prediction_df['predictions_b'] = denormalize(prediction_df["predictions_b"].values ,-48,96)
prediction_df['predictions_c'] = denormalize(prediction_df["predictions_c"].values , -792,600)

In [ ]:
prediction_df.head(100)

# Saving the model

In [ ]:
torch.save(model.state_dict(),"./save.pth")

In [ ]:
model_test = Quadratic_Model()
model_test.load_state_dict(torch.load("./save.pth"))

In [ ]:
model_test.eval()
random_tensor = torch.ones(1,3,256,256)
model_test(random_tensor)